Notes:
 * run 17 on the TOI targets

In [26]:
import os

chkpt_root = '/mnt/tess/astronet/checkpoints/extended_25_run_17'
data_files = '/mnt/tess/astronet/tfrecords-toi/*'
tces_file = '/mnt/tess/astronet/tces-toi.csv'

nruns = 10

def load_ensemble(chkpt_root, nruns):
    checkpts = []
    for i in range(nruns):
        parent = os.path.join(chkpt_root, str(i + 1))
        if not os.path.exists(parent):
            break
        all_dirs = os.listdir(parent)
        if not all_dirs:
            break
        d, = all_dirs
        checkpts.append(os.path.join(parent, d))
    return checkpts

paths = load_ensemble(chkpt_root, nruns)
paths

['/mnt/tess/astronet/checkpoints/extended_25_run_17/1/AstroCNNModel_extended_20210321_154527',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/2/AstroCNNModel_extended_20210321_161606',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/3/AstroCNNModel_extended_20210321_164646',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/4/AstroCNNModel_extended_20210321_171723',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/5/AstroCNNModel_extended_20210321_174729',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/6/AstroCNNModel_extended_20210321_181710',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/7/AstroCNNModel_extended_20210321_184652',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/8/AstroCNNModel_extended_20210321_191655',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/9/AstroCNNModel_extended_20210321_194703',
 '/mnt/tess/astronet/checkpoints/extended_25_run_17/10/AstroCNNModel_extended_20210321_201726']

In [27]:
import getpass
import os
from astronet import predict
import tensorflow as tf


def run_predictions(path):
    predict.FLAGS = predict.parser.parse_args([
      '--model_dir', path,
      '--data_files', data_files,
      '--output_file', '',
    ])

    return predict.predict()


paths = load_ensemble(chkpt_root, nruns)
ensemble_preds = []
config = None
for i, path in enumerate(paths):
    print(f'Running model {i + 1}')
    preds, config = run_predictions(path)
    ensemble_preds.append(preds)
    print()

Running model 1
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 2
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 3
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 4
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 5
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 6
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 7
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 8
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 9
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records
Running model 10
Binary prediction threshold: 0.2152499407880693 (orientative)
2041 records


In [28]:
labels = ['disp_E', 'disp_N', 'disp_J', 'disp_S', 'disp_B']

col_e = labels.index('disp_E')
# thresh = config.hparams.prediction_threshold
# thresh = 0.030485098838860747  # From the validation numbers - maximum thrershold for 100% recall
thresh = 0.31245827674871207  # Relaxed to match Liang's precision value

In [29]:
import pandas as pd

det_preds = ensemble_preds[0]

for i, pred in enumerate(ensemble_preds):
    tmp = pred[labels]
    newl = [l[-1] + str(i) for l in labels]
    for l, nl in zip(labels, newl):
        tmp[nl] = tmp[l]
    tmp = tmp[newl]
    det_preds = pd.concat([
        det_preds,
        tmp
    ], axis=1)
    
final_order = ['tic_id']
for i in range(len(ensemble_preds)):
    final_order += ['E' + str(i)]
for i in range(len(ensemble_preds)):
    final_order += ['N' + str(i)]
for i in range(len(ensemble_preds)):
    final_order += ['J' + str(i)]
for i in range(len(ensemble_preds)):
    final_order += ['S' + str(i)]
for i in range(len(ensemble_preds)):
    final_order += ['B' + str(i)]

det_preds = det_preds[final_order]

In [30]:
det_preds.head()

,tic_id,E0,E1,E2,E3,E4,E5,E6,E7,E8,...,B0,B1,B2,B3,B4,B5,B6,B7,B8,B9
0,176860064,0.998947,0.995673,0.987614,0.97779,0.991777,0.997504,0.998031,0.996365,0.995827,...,0.00026,0.000439,0.000777,0.001183,0.000873,0.000383,0.000349,0.000781,0.000391,0.000444
1,236887394,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,237086564,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,243185500,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,16920150,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
# det_preds.to_csv('~/Astronet-Triage/detailed-preds-toi.csv')

In [32]:
import numpy as np
import pandas as pd

agg_preds = {}
tic_ids = {}

pred_idx = 0

for preds in ensemble_preds:
    for row_id in preds.index:
        if row_id not in agg_preds:
            agg_preds[row_id] = []
            tic_ids[row_id] = preds['tic_id'][row_id]

        row = preds[preds.index == row_id]
        pred_v = row.values[pred_idx]
        if len(row.values) > 1:
            print(f'Warning: duplicate predictions for {row_id}')
        if pred_v[col_e] >= thresh:
            agg_preds[row_id].append('disp_E')
        else:
            masked_v = [v if i != col_e else 0 for i, v in enumerate(pred_v)]
            agg_preds[row_id].append(preds.columns[np.argmax(masked_v)])

In [33]:
final_preds = []
for row_id in list(agg_preds.keys()):
    counts = {l: 0 for l in labels}
    for e in agg_preds[row_id]:
        counts[e] += 1
    maxcount = max(counts.values())
    counts.update({
        'row_id': row_id,
        'tic_id': tic_ids[row_id],
        'maxcount': maxcount,
    })
    final_preds.append(counts)

final_preds = pd.DataFrame(final_preds).set_index('tic_id')

In [34]:
final_preds.head()

,disp_E,disp_N,disp_J,disp_S,disp_B,row_id,maxcount
tic_id,,,,,,,
176860064,10,0,0,0,0,0,10
236887394,10,0,0,0,0,1,10
237086564,10,0,0,0,0,2,10
243185500,10,0,0,0,0,3,10
16920150,10,0,0,0,0,4,10


In [47]:
def compare(ensemble_preds, filter):
    result = ensemble_preds[0][filter]
    for preds in ensemble_preds[1:]:
        result = result.append(preds[filter])
    return result

compare(ensemble_preds, preds.tic_id == 146261607).sort_index()

,tic_id,disp_E,disp_N,disp_J,disp_S,disp_B
1144,146261607,0.001933,0.057857,0.985893,0.010260,0.081816
1144,146261607,0.003601,0.071325,0.972887,0.035778,0.008942
1144,146261607,0.002979,0.064766,0.988681,0.006367,0.014174
1144,146261607,0.001370,0.136906,0.992757,0.002493,0.082827
1144,146261607,0.000294,0.078095,0.998486,0.001020,0.032257
1144,146261607,0.001926,0.128329,0.987459,0.009631,0.018141
1144,146261607,0.004094,0.108786,0.985708,0.010657,0.015761
1144,146261607,0.001130,0.058510,0.994949,0.005938,0.010631
1144,146261607,0.030710,0.154052,0.868153,0.082078,0.028386
1144,146261607,0.003636,0.109136,0.984633,0.010141,0.045070


In [36]:
# final_preds = final_preds.join(
#     pd.read_csv(tces_file, header=0, low_memory=False),
#     on='row_id',
#     how='inner'
# )

In [37]:
# final_preds.to_csv('~/Astronet-Triage/preds-toi.csv')